In [27]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import faiss
import torch
from tqdm.auto import trange
from string import punctuation
from spacy.lang.en import stop_words
import spacy
import pickle
import re
import torch.functional as F

def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request_csqa(x):
    return f"{x['question_state']} {x['question']}"

def parse_request_dream(x):
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}" # 

def match_pairs(c1,c2):
    state = list(c1)+list(c2)
    idx = np.unique(state, return_index=True)[1]
    return [state[i] for i in sorted(idx)][::-1]

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    
def cls_pooling(model_output):
    return model_output[0][:, 0]

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=1024):
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(
            texts[i*batch_size:(i+1)*batch_size],
            truncation=True,
            padding='longest',
            max_length=max_length,
            return_tensors='pt'
        )
        encs['input_ids'] = encs['input_ids'].cuda()
        encs['attention_mask'] = encs['attention_mask'].cuda()
        if 'token_type_ids' in encs.keys():
            encs['token_type_ids'] = encs['token_type_ids'].cuda()
        model_output = model(**encs)
        embeds = cls_pooling(model_output)#mean_pooling(model_output, encs['attention_mask'])
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

class TextProcessor():
    def __init__(self, kn_base='en_core_web_sm'):
        self.nlp = spacy.load(kn_base)
        self.stop_wrods = stop_words.STOP_WORDS | set(punctuation) | set('123456789')
        
    def prepare_text(self, s):
        s = s.lower().replace('_',' ').strip()
        for p in punctuation:
            s = s.replace(p,' ')
        return re.sub(" +", " ",s)
    
    def precess_text(self,s):
        s = self.prepare_text(s)
        doc = self.nlp(s)
        lemms = [token.lemma_ for token in doc]
        lemms = [token for token in lemms if token not in self.stop_wrods]
        return lemms
    
    def process_batch(self,texts,verbose=True):
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.precess_text,iterator)
        return prep_texts

def rank_by_idf(texts,idf_matrix,topk=5):
    idx = np.unique(texts, return_index=True)[1]
    texts = [texts[i] for i in sorted(idx)]
    idfs = [(idf_matrix[x] if x in idf_matrix else 0) for x in texts]
    return [texts[i] for i in np.argsort(idfs)[:topk]]

def setup_index(path):   
    features = pickle.load(open(path,'rb'))
    index = faiss.IndexFlatIP(features.shape[1]) #use faiss.IndexFlatL2 for l2 distance
    faiss.normalize_L2(features)
    index.add(features)
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)
    return index

def get_retrivers():
    encoder = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5').cuda()
    tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
    return encoder,tokenizer

class EntityLinker():
    def __init__(self, kg, text_proc, prepared2concept, prepared_maper, clear_nodes):
        self.kg = kg
        self.text_proc = text_proc
        self.prepared2concept = prepared2concept
        self.prepared_concepts = prepared_maper
        self.cleared_nodes = clear_nodes
        
    def link_sentence(self,s):
        candidates = []
        s = self.text_proc.precess_text(s)
        s_st = set(s)
        for i,k in enumerate(self.kg):
            if len(k) == 0:
                continue
            elif len(k) == 1:
                if k[0] in s_st:
                    candidates.append(i)
            else:
                if all(x in s_st for x in k):
                    idxes = [s.index(x) for x in k]
                    if all(idxes[j+1] - idxes[j] == 1 for j in range(len(k)-1)):
                           candidates.append(i)
        return candidates
    
    def batch_linking(self,texts,verbose=True):
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.link_sentence,iterator)
        return prep_texts
    
    def get_concepts(self,ids):
        ids = [self.kg[i] for i in ids] # from index to lemms
        ids = [' '.join(x) for x in ids]
        return [self.prepared2concept[i] for i in ids] # from lemms to concepts
    
    
class SampleNodesMatcher():
    def __init__(self, encoder, tokenizer, index, idf_matrix, maper, linker):
        self.encoder = encoder
        self.tokenizer = tokenizer
        self.index = index
        self.idf_matrix = idf_matrix
        self.linker = linker
        self.maper = maper
    
    def __call__(self, sample,type='csqa',use_pretty=False):
        if type == 'csqa':
            req = parse_request_csqa(sample)
        else:
            req = parse_request_dream(sample)
        if use_pretty:
            maper = self.linker.cleared_nodes
        else:
            maper = self.maper
        embeds = create_embeds([req], self.encoder, self.tokenizer,batch_size=2,max_length=512)
        faiss.normalize_L2(embeds)
        D, I = self.index.search(embeds,k=7)
        I = [maper[i] for i in I[0]]
        
        linker_cand = self.linker.link_sentence(req)
        linker_cand = self.linker.get_concepts(linker_cand)
        linker_cand = rank_by_idf(linker_cand,self.idf_matrix[type])
        if type == 'csqa':
            return match_pairs(linker_cand,I)
        else:
            choice_embeds = create_embeds([sample['choice_0'], sample['choice_1'], sample['choice_2']], self.encoder, self.tokenizer,batch_size=16,max_length=512)
            faiss.normalize_L2(choice_embeds)
            D, I_q = self.index.search(choice_embeds,k=1)
            choice_encs = [self.linker.cleared_nodes[i] for i in I_q[:,0]]
            return match_pairs(linker_cand,I), choice_encs

def setup_processor(linker_path="entity_linker.pkl", embeds_path='faiss_embeds.pkl', idf_path='idf_matrix.pkl'):
    linker = pickle.load(open(linker_path,'rb'))
    index = setup_index(embeds_path)
    idf_matrix = pickle.load(open(idf_path,'rb'))
    encoder,tokenizer = get_retrivers()
    return SampleNodesMatcher(
    encoder, tokenizer, index, idf_matrix, linker.maper_inv, linker
    )

In [28]:
proc = setup_processor()

In [16]:
text_processor = TextProcessor()

In [17]:
prepared_concepts =  pickle.load(open('conceprs_enc.pkl','rb'))

In [18]:
prepared2concept = pickle.load(open('prepared2concept.pkl','rb'))

In [19]:
graph_state = pickle.load(open('graph.pkl', 'rb'))

In [23]:
clear_nodes = pickle.load(open('cleared_concepts.pkl', 'rb'))

In [24]:
linker = EntityLinker(prepared_concepts, text_processor, prepared2concept, graph_state['concepts_maper'],clear_nodes)

In [25]:
linker.maper_inv = graph_state['concepts_maper_inv']

In [26]:
with open("entity_linker.pkl",'wb') as f:
    pickle.dump(linker,f)

In [27]:
index = setup_index('faiss_embeds.pkl')

In [28]:
encoder, tokenizer = get_retrivers()

In [29]:
idf_matrix = {}
idf_csqa = pickle.load(open('idf_csqa.pkl','rb'))
idf_dream = pickle.load(open('idf_dream.pkl','rb'))
idf_dream = {graph_state['concepts_maper'][u]:v for u,v in idf_dream.items()}
idf_matrix['csqa'] = idf_csqa
idf_matrix['dream'] = idf_dream

In [4]:
procc = SampleNodesMatcher(
    encoder, tokenizer, index, idf_matrix, graph_state['concepts_maper_inv'], linker
)

NameError: name 'encoder' is not defined

In [31]:
def make_df(path,is_test=False):
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question_state'] = data['question'].apply(lambda x: x['question_concept'])
    df['id'] = data['id']
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = -100
    return df
df = make_df('csqa_data/train.statement.jsonl')

In [32]:
def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_dialog(sentences):
    return '; '.join(sentences)

def parse_request(x):
    return f"{x['question_state']} {x['question']}"

def make_dream_df(path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

dream_df = make_dream_df('data/train.statement.jsonl')

In [33]:
cands_csq = pickle.load(open('csqa_data/all_candsV3.pkl','rb'))

In [34]:
cands_csq['train_all_cands'][1]

['will_go_somewhere',
 'waste_gasoline',
 'wanting_to_go_somewhere',
 'would_get_to_somewhere',
 'j_walkers',
 'wanted_to_get_to_work',
 'people',
 'want',
 'sammy']

In [35]:
sample_csqa = {
    "question_state": "i",
    "question": 'Kostya Sigaylov'
}

In [37]:
proc(df.iloc[1])

  0%|          | 0/1 [00:00<?, ?it/s]

['will_go_somewhere',
 'waste_gasoline',
 'wanting_to_go_somewhere',
 'would_get_to_somewhere',
 'j_walkers',
 'wanted_to_get_to_work',
 'sammy',
 'want',
 'people']

In [37]:
dream_df

dialog_id                                             dialog  \
0             0  W: Well, I'm afraid my cooking isn't to your t...   
1             0  W: Well, I'm afraid my cooking isn't to your t...   
2             1  M: Look at the girl on the bike!; F: Oh, yes s...   
3             2  M: Can you accompany me while I am singing.; W...   
4             3  W: I can't figure out what's wrong with my wat...   
...         ...                                                ...   
7302       4637  W: I've been reading about a new snack food. I...   
7303       4637  W: I've been reading about a new snack food. I...   
7304       4637  W: I've been reading about a new snack food. I...   
7305       4637  W: I've been reading about a new snack food. I...   
7306       4638  W: Do people drive on the right in England?; M...   

                                               question  \
0       What does the man think of the woman's cooking?   
1                What does the woman ask the man to do?   
2                            Where are the two persons?   
3               What does the man want the woman to do?   
4                               What does the man mean?   
...                                                 ...   
7302  Which of the following is NOT mentioned in the...   
7303  What would be a good reason for the man to buy...   
7304  According to the woman, what is an advantage o...   
7305            When will the new product be available?   
7306                       What are they talking about?   

                                               choice_0  \
0                                 It's really terrible.   
1                               Improve cooking skills.   
2                                               At home   
3                                    To play the piano.   
4     It will take about one month to repair the watch.   
...                                                 ...   
7302                                            Tomato.   
7303                            Its low purchase price.   
7304                  Its bland flavoring is healthful.   
7305                                    In a few weeks.   
7306                                Driving in England.   

                                     choice_1  \
0                      It's very good indeed.   
1                   Bring his wife next time.   
2                          In their classroom   
3                      To give a performance.   
4     The watch is no longer worth repairing.   
...                                       ...   
7302                                  Cheese.   
7303                   Its wide availability.   
7304            It can be stored a long time.   
7305                     In about two months.   
7306                      How to drive a car.   

                                               choice_2  labels  
0                        It's better than what he does.       1  
1                                       Have some soup.       1  
2                                         On the street       2  
3                                       To sing a song.       0  
4              It is a good idea to keep the old watch.       1  
...                                                 ...     ...  
7302                                              Fish.       2  
7303                        Its good nutritional value.       2  
7304                     It goes well with fish dishes.       1  
7305                                In about two years.       2  
7306  Whether to have the right to drive a car in En...       0  

[7307 rows x 7 columns]

In [38]:
dream_state = pickle.load(open('dream_all_candidates.pkl','rb'))

In [72]:
pd.read_csv("prep_data/train_choices.csv")

choice_1_concept               choice_2_concept  \
0                  pretty awful                      very good   
1     practising cooking skills                    put wife in   
2                       at home                  in classrooms   
3                 playing piano                 for performing   
4                  broken watch                   broken watch   
...                         ...                            ...   
7302                     tomato                         cheese   
7303                 low prices           greater availability   
7304                 bland diet  storing food for long periods   
7305                   in weeks                      in months   
7306          driving somewhere          know how to drive car   

           choice_3_concept  
0                its better  
1               put soup in  
2                 on street  
3         singing something  
4          old pocket watch  
...                     ...  
7302              fish fish  
7303         good nutrition  
7304       nice fish dinner  
7305               two year  
7306  required to drive car  

[7307 rows x 3 columns]

In [18]:
dream_df.iloc[0]

dialog_id                                                    0
dialog       W: Well, I'm afraid my cooking isn't to your t...
question       What does the man think of the woman's cooking?
choice_0                                 It's really terrible.
choice_1                                It's very good indeed.
choice_2                        It's better than what he does.
labels                                                       1
Name: 0, dtype: object

In [7]:
procc(dream_df.iloc[7305],type='dream')

NameError: name 'procc' is not defined